Neuroevolution: Exercise 1 
=========
###### Artur Ganzha 10019651
---------	
###### Raul Gorek 10061333
---------	

In [55]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

## Aufgabe 1

In [56]:
# Datensatz
X = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
y = np.array([0.0, 1.0, 1.0, 0.0]) 

In [57]:
def relu(x):
    return x * (x > 0)

In [58]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

## Aufgabe 2

In [59]:
class NeuralNetwork:
    def __init__(self, layers: list):
        self.layers = layers
        for layer in self.layers:
            layer.init()

    def forward_pass(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward_pass(self, deriv, lr):
        for layer in reversed(self.layers):
            deriv = layer.backward(deriv, lr)

In [60]:
class Linear:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
    
    def init(self):
        self.W = np.random.uniform(-1, 1,(self.input_size,self.output_size))
        self.B = np.zeros((1, self.output_size))
    
    def forward(self, x):
        self.fw = x
        return np.dot(x, self.W) + self.B
    
    def backward(self, d, lr):
        d_w = np.dot(self.fw.T, d)
        d_e = np.dot(d, self.W.T)
        d_b = np.sum(d, axis=0, keepdims=True)
        self.W -= lr * d_w / self.fw.shape[0]
        self.B -= lr * d_b / self.fw.shape[0]
        return d_e

In [61]:
class ReLU:
    def __init__(self):
        pass

    def init(self):
        pass

    def forward(self, x):
        self.fw = x
        return x * (x > 0)
    
    def backward(self, d, lr):
        return d * np.where(self.fw > 0, 1.0, 0.0)

In [62]:
class Sigmoid:
    def __init__(self):
        pass
    
    def init(self):
        pass
    
    def forward(self, x):
        self.fw = x
        self.out = 1.0 / (1.0 + np.exp(-x))
        return self.out
    
    def backward(self, d, lr):
        return d * (self.out * (1.0 - self.out))

In [63]:
arch = [
    Linear(2, 8),
    ReLU(),
    Linear(8,1),
    Sigmoid()
]

In [64]:
neural_net = NeuralNetwork(arch)

In [65]:
batch = np.array([[0,0],[0,1]])
neural_net.forward_pass(batch)

array([[0.5       ],
       [0.21148131]])

## Aufgabe 3

In [66]:
def binary_cross_entropy_loss(prediction, ground_truth):
    return -(ground_truth * np.log(prediction) + (1-ground_truth) * np.log(1-prediction))

In [67]:
def derivative_bcel(prediction, ground_truth):
    x =  np.where(ground_truth == 0, 1.0 / (1.0 - prediction), -1.0 / prediction)
    return x

In [68]:
def derivative_relu(z):
    return np.where(z > 0, 1.0, 0.0)

In [69]:
def derivative_sigmoid(z):
    x = sigmoid(z)
    return x * (1-x)

# Aufgabe 4

#### Siehe backward-Methoden in den Klassen aus Aufgabe 2

## Aufgabe 5

In [70]:
from IPython.display import clear_output

In [71]:
def train(neural_net: NeuralNetwork, learn_rate, batch_size, num_epochs):
    accuracies = []
    bar = tqdm(range(num_epochs), desc="Training")
    for i in bar:
        batch = np.random.randint(0,2,size=(batch_size, 2))
        labels = np.array([[i[0] ^ i[1]]for i in batch]).astype(float)
        y_hat = neural_net.forward_pass(batch)
        loss = binary_cross_entropy_loss(y_hat, labels)
        deriv_loss = derivative_bcel(y_hat, labels)
        neural_net.backward_pass(deriv_loss, learn_rate)

        accuracy = np.mean(1 - np.abs(labels - y_hat))
        accuracies.append(accuracy)
        bar.set_postfix(Loss=np.mean(loss), Accuracy=accuracy)

In [72]:
epochs_tqdm = 2_000
batch_size = 10
learn_rate = 0.05
neural_net = NeuralNetwork(arch)
train(neural_net, learn_rate, batch_size, epochs_tqdm)
batch = np.array([[0,0],[0,1],[1,0],[1,1]])
print(neural_net.forward_pass(batch))

Training: 100%|██████████| 2000/2000 [00:00<00:00, 2201.47it/s, Accuracy=0.982, Loss=0.0181] 

[[0.02889326]
 [0.98842162]
 [0.98954518]
 [0.00757111]]
